In [1]:
library(dplyr)
library(zoo)
library(ggplot2)
library(seqminer)
library(EnsDb.Hsapiens.v86)
library(ggbio)
library(patchwork)
library(stringr)

plot_region_pileups = function(chrom, # chromosome
                                start, # start coordinate
                                end, # end coordinate
                                bam_bed_file, # tabix indexed BED file with chrom,start,end,cell
                                grouping, # dataframe with cell,group,total_reads
                                ensdb=EnsDb.Hsapiens.v86, # EnsDb to allow display of genes
                                smoothing_window=100, # window to use for averaging
                                x_steps=3000) { # limits number of actual to plot data for

  # Validate the grouping dataframe
  if (! all(c('cell', 'group', 'total_reads') %in% colnames(grouping))) {
    stop('Grouping DF must have cell, grouping, and total_reads columns.')
  }

  # Get reads for the requested region of BED file (using tabix)
  range_string = paste0(chrom, ':', start, '-', end)
  reads = seqminer::tabix.read.table(tabixRange=range_string, tabixFile=bam_bed_file)
  colnames(reads) = c('chrom', 'start', 'stop', 'cell', 'number')

  # Get count of cells per group for normalization
  grouping = grouping %>%
    group_by(group) %>%
    mutate(cells_in_group = n()) %>%
    ungroup()

  # Restrict to cell barcodes specified by user
  reads = dplyr::filter(reads, cell %in% grouping$cell)
  reads = dplyr::inner_join(reads, grouping, by='cell')

  # Expand reads so have one line per base
  expanded_ranges = dplyr::bind_rows(lapply(1:nrow(reads), function(i) {
    interval = reads[i, 'start']:reads[i, 'stop']
    return(data.frame(position=interval,
                      value=1,
                      cell=reads[i, 'cell'],
                      group=reads[i, 'group'],
                      scaling_factor=reads[i, 'total_reads'] ,
                      cells_in_group=reads[i, 'cells_in_group']))
  }))

  # Apply scaling factor and account for different cell counts
  # Also apply windowed smoothing
  expanded_ranges = expanded_ranges %>%
    group_by(position, group) %>%
    summarize(total=sum(value / scaling_factor) / mean(cells_in_group)) %>%
    group_by(group) %>%
    arrange(position) %>%
    mutate(total_smoothed=zoo::rollapply(total, smoothing_window, mean, align='center',fill=NA)) %>%
    ungroup()

  # Finally, restrict to x_steps points for plotting to speed things up
  total_range = end - start
  stepsize = ceiling(total_range / x_steps)
  positions_to_show = seq(start, end, by=stepsize)
  expanded_ranges = subset(expanded_ranges, position %in% positions_to_show)

  # Make plots faceted by group
  max_value = max(expanded_ranges$total_smoothed, na.rm = TRUE)

  plot_object = ggplot() +
    geom_bar(data=expanded_ranges, aes(position, total_smoothed), stat='identity', color='black', fill="#d3d3d3", alpha=0.3) +
    facet_wrap(~group, ncol=1, strip.position="right") +
    ylim(0, max_value * 1.4) +
    xlab('Position (bp)') +
    ylab(paste0('Normalized Signal (', smoothing_window, ' bp window)')) +
    theme_classic()

  # Add gene track using ensembl ID
  gr <- GRanges(seqnames = str_replace(chrom, 'chr', ''), IRanges(start, end), strand = "*")
  filters = AnnotationFilterList(GRangesFilter(gr), GenebiotypeFilter('protein_coding'))

  genes = autoplot(ensdb, filters, names.expr = "gene_name")

  genes_plot = genes@ggplot +
    xlim(start, end) +
    theme_classic()

  # Now combine the two plots into one (using patchwork)
  return(plot_object / (genes_plot + xlim(start, end)))
}


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Loading required package: ensembldb

Loading required package: BiocGenerics


Attaching package: ‘BiocGenerics’


The following objects are masked from ‘package:dplyr’:

    combine, intersect, setdiff, union


The following objects are masked from ‘package:stats’:

    IQR, mad, sd, var, xtabs


The following objects are masked from ‘package:base’:

    anyDuplicated, append, as.data.frame, basename, cbind, colnames,
    dirname, do.call, duplicated, eval, evalq, Filter, Find, get, grep,
    grepl, intersect, is.unsorted, lapply, Map, mapply, match, mget,
    order, paste, pmax, pmax.int, pmin, pmin.int, Position, rank,
    rbind, Reduce, rownames, sapply, setdiff, sort, tabl

In [12]:
dev.off()

pdf 
  3

In [32]:
grouping_table = read.delim('subgroup_annotation_for_R.txt', sep='\t')
pdf(file="TFBS_HNF4A_ALB_III_20230125.pdf")

plot_region_pileups('chr4', 73415930, 73416346, 'fragments.tsv.gz', grouping=grouping_table, smoothing_window=10, x_steps = 10000)
dev.off()

`summarise()` has grouped output by 'position'. You can override using the
`.groups` argument.
Warning message:
"'GenebiotypeFilter' is deprecated.
Use 'GeneBiotypeFilter' instead.
See help("Deprecated")"
Fetching data...
OK

Parsing exons...
OK
Defining introns...
OK
Defining UTRs...
OK
Defining CDS...
OK

aggregating...

Done

Constructing graphics...

Coordinate system already present. Adding new coordinate system, which will replace the existing one.

Warning message:
"Removed 23 rows containing missing values (position_stack)."


png 
  2